# Loading JSON files

## Crude Oil

Let us reconsider the oil-model from the beginning of the course.

$$\begin{array}{rll}
 \text{min} & 3x+5y \\
 \text{s.t.} & 2x+y \ge 3 \\
 & 2x + 2y \ge 5 \\
 & x + 4y \ge 4 \\
 & x,y \ge 0 \\
\end{array}
$$

In the old implementation of this model, we defined our data within the code. This does not create a clear separation between data and code, and thus the model is less flexible, and it is harder to switch datasets. In the next step, we want to improve on this disadvantage and load our data from `crudeoil_data.json`.

### Step 1: Loading Modules

In [ ]:
from gurobipy import *
import json


### Step 2: Create a Model

In [ ]:
m = Model()

### Step 3: Load the Data

In [ ]:
with open("../Excercises/crudeoil_data.json") as json_file:
    data = json.load(json_file)

data

### Step 4: Create Variables and set Objective Function

In [ ]:
x = m.addVars(data["processes"],obj=data["process_cost"],lb=0,vtype=GRB.CONTINUOUS,name="process")

### Step 5: Add Constraints

In [ ]:
for ot in data["oil_types"]:
    m.addConstr(x.prod(data["production"][ot])>=data["demand"][ot])

### Step 6: Solve

In [ ]:
m.optimize()

### Step 7: Display Solution

In [ ]:
for v in m.getVars():
    print('%g %s' % (v.x, v.varName))

print('Cost: %g' % m.objVal)


## Carsharing

**a)** Reimplement the carsharing model using the file `carsharing_data1.json`. Be sure to use all of the information given in the `json` file and make your implementation as flexible as possible.

**b)** Check if your implementation also works on `carsharing_data2.json`.

In [ ]:
from gurobipy import *
from math import sqrt
import json

def dist(positions,i,j): #distance function
    return data["distanceFactor"] * math.sqrt((positions[i][0]-positions[j][0])**2+(positions[i][1]-positions[j][1])**2)

m = Model() #create model

with open("../Excercises/carsharing_data2.json") as json_file:
    data = json.load(json_file)

flow=m.addVars(data["zones"],data["zones"],vtype=GRB.CONTINUOUS,lb=0) #create variables
for i in range(data["zones"]): #add constraints
    expr=0
    for j in range(data["zones"]):
        expr+=flow[(j,i)]
        expr-=flow[(i,j)]
    m.addConstr(expr>=data["needed"][i]-data["available"][i])
    
for i in range(data["zones"]): #add objective
    for j in range(data["zones"]):
        flow[(i,j)].obj=data["costPerKm"]*dist(data["positions"],i,j)
        
m.optimize() #optimize

for i in range(data["zones"]): #report result
    print(f"outflow from station {i}: {[flow[(i,j)].x for j in range(data['zones'])]}")